In [1]:
# Cell 1: Install necessary libraries
!pip install -q tensorflow kaggle scikit-learn

print("Libraries installed successfully.")

Libraries installed successfully.


In [2]:
# Cell 2: Import required libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import os
import shutil
from google.colab import files, drive
import numpy as np

print("Libraries imported successfully.")

Libraries imported successfully.


In [3]:
# Cell 3: Handle Kaggle API key upload and download dataset
kaggle_file_path = '/content/kaggle.json'
if not os.path.exists(kaggle_file_path):
    # Upload Kaggle API key if it doesn't exist
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        # Configure Kaggle API key
        !mkdir -p ~/.kaggle
        !cp /content/kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
        print("kaggle.json configured.")

# Download the dataset from Kaggle
!kaggle datasets download -d asdasdasasdas/garbage-classification
# Unzip the downloaded dataset
!unzip -o garbage-classification.zip -d /content/kaggle_data
kaggle_data_path = '/content/kaggle_data/Garbage classification/Garbage classification'
print("Kaggle dataset downloaded and unzipped successfully.")

Streaming output truncated to the last 5000 lines.
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard153.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard154.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard155.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard156.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard157.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard158.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard159.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classification/cardboard/cardboard16.jpg  
  inflating: /content/kaggle_data/Garbage classification/Garbage classificatio

In [4]:
# Cell 4: Organize and reclassify dataset
biodegradable = ['paper']
non_biodegradable = ['plastic', 'trash']
recyclable = ['cardboard', 'glass', 'metal']
base_dir = '/content/reorganized_kaggle_data'
os.makedirs(base_dir, exist_ok=True)

def move_images_to_new_category(original_category, new_category):
    # Move images from original category to a new category directory
    source_dir = os.path.join(kaggle_data_path, original_category)
    target_dir = os.path.join(base_dir, new_category)
    os.makedirs(target_dir, exist_ok=True)

    if os.path.exists(source_dir):
        for img_file in os.listdir(source_dir):
            src_file = os.path.join(source_dir, img_file)
            dst_file = os.path.join(target_dir, img_file)

            # Skip if the file already exists in the target directory
            if os.path.exists(dst_file):
                print(f"File {dst_file} already exists, skipping.")
                continue

            shutil.move(src_file, dst_file)

# Move images to the appropriate new categories
for category in biodegradable:
    move_images_to_new_category(category, 'biodegradable')
for category in non_biodegradable:
    move_images_to_new_category(category, 'non_biodegradable')
for category in recyclable:
    move_images_to_new_category(category, 'recyclable')

print("Dataset reorganization complete.")

Dataset reorganization complete.


In [5]:
# Cell 5: Mount Google Drive to access additional datasets
drive.mount('/content/drive')
google_drive_path = '/content/drive/MyDrive/CSRP/dataSet'
merged_dataset_dir = '/content/merged_dataset'
os.makedirs(merged_dataset_dir, exist_ok=True)

def copy_images(category, source_dir, target_dir):
    # Copy images from the source directory to the target directory for a specific category
    source_category_dir = os.path.join(source_dir, category)
    target_category_dir = os.path.join(target_dir, category)
    os.makedirs(target_category_dir, exist_ok=True)
    if os.path.exists(source_category_dir):
        for img_file in os.listdir(source_category_dir):
            shutil.copy(os.path.join(source_category_dir, img_file), target_category_dir)

# Copy images from Google Drive and reorganized Kaggle dataset to the merged dataset directory
for category in ['biodegradable', 'non_biodegradable', 'recyclable']:
    copy_images(category, google_drive_path, merged_dataset_dir)
    copy_images(category, base_dir, merged_dataset_dir)

print("Merged dataset created successfully.")

Mounted at /content/drive
Merged dataset created successfully.


In [6]:
# Cell 6: Define data augmentation parameters for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    validation_split=0.2
)

# Generate training data
train_generator = train_datagen.flow_from_directory(
    merged_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Generate validation data
validation_generator = train_datagen.flow_from_directory(
    merged_dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("Optimized data generators set up successfully.")

Found 2107 images belonging to 3 classes.
Found 525 images belonging to 3 classes.
Optimized data generators set up successfully.


In [7]:
# Cell 7: Calculate class weights for handling imbalances
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

class_weights = dict(enumerate(class_weights))
print("Class weights computed successfully.")

Class weights computed successfully.


In [8]:
# Cell 8: Build MobileNetV3 Small model
base_model = MobileNetV3Small(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = True  # Allow fine-tuning of the entire model

x = GlobalAveragePooling2D()(base_model.output)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print("MobileNetV3 Small model built and compiled successfully.")

4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MobileNetV3 Small model built and compiled successfully.


In [9]:
# Cell 9: Define learning rate scheduler and callbacks
def lr_scheduler(epoch, lr):
    return lr * 0.5 if epoch > 10 else lr

callbacks = [
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.000001, verbose=1),
    LearningRateScheduler(lr_scheduler),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False, verbose=1)
]

print("Callbacks set up successfully.")

Callbacks set up successfully.


In [10]:
# Cell 10: Train the model
history = model.fit(
    train_generator,
    epochs=50,  # Increase the number of epochs for better learning
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks
)

print("Model training complete.")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.4214 - loss: 1.4661 - val_accuracy: 0.5105 - val_loss: 1.1550 - learning_rate: 1.0000e-04
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.5409 - loss: 1.0822 - val_accuracy: 0.5105 - val_loss: 1.3454 - learning_rate: 1.0000e-04
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.6287 - loss: 0.8737 - val_accuracy: 0.5105 - val_loss: 1.5303 - learning_rate: 1.0000e-04
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.6589 - loss: 0.8153 - val_accuracy: 0.5105 - val_loss: 1.5356 - learning_rate: 1.0000e-04
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.6829 - loss: 0.7760 - val_accuracy: 0.5105 - val_loss: 1.3548 - learning_rate: 1.0000e-04
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.7261 - loss: 0.6795 - val_accuracy: 0.5105 - val_loss: 1.2588 - learning_rate: 1.0000e-04
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 196s 3s/step - accuracy: 0.7367 - loss: 0.6

In [11]:
# Cell 11: Convert model to TensorFlow Lite
checkpoint_path = 'best_model.keras'
if os.path.exists(checkpoint_path):
    model.load_weights(checkpoint_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    with open('waste_classifier.tflite', 'wb') as f:
        f.write(tflite_model)

    print("Model successfully converted to TensorFlow Lite.")
else:
    print(f"Model checkpoint '{checkpoint_path}' not found.")

Saved artifact at '/tmp/tmp7uhqxz9f'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  133460431641536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460431645584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460431646640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460431643472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460431642240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460447052896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460447052192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460447056768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460447054304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133460447054656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13346044706134

In [12]:
# Cell 12: Download the TensorFlow Lite model
files.download('waste_classifier.tflite')
print("TensorFlow Lite model downloaded successfully.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

TensorFlow Lite model downloaded successfully.
